##### Copyright 2025 Google LLC.

In [33]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: JSON Mode Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

The Gemini API can be used to generate a JSON output if you set the schema that you would like to use.

Two methods are available. You can either set the desired output in the prompt or supply a schema to the model separately.

### Install dependencies

In [34]:
%pip install -U -q "google-genai"

In [35]:
from google import genai
from pydantic import BaseModel

### Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [36]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

## Set your constrained output in the prompt

 Activate JSON mode by specifying a response_schema and setting `response_mime_type="application/json" ` users can constrain the model to produce a JSON response following a given structure.

For this first example just describe the schema you want back in the prompt:

In [37]:
prompt = """List a few popular cookie recipes using this JSON schema:

Recipe = {'recipe_name': str}
Return: list[Recipe]"""

In [38]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt,
    config={
        'response_mime_type': 'application/json',
    },
 )

Parse the string to JSON:

In [39]:
response.parsed
print(response.text)

[
  {
    "recipe_name": "Chocolate Chip Cookies"
  },
  {
    "recipe_name": "Peanut Butter Cookies"
  },
  {
    "recipe_name": "Sugar Cookies"
  },
  {
    "recipe_name": "Oatmeal Raisin Cookies"
  },
  {
    "recipe_name": "Snickerdoodles"
  }
]


## Supply the schema to the model directly

The newest models (1.5 and beyond) allow you to pass a schema object (or a python type equivalent) directly and the output will strictly follow that schema.

Following the same example as the previous section, here's that recipe type:

In [40]:
class Recipe(BaseModel):
    recipe_name: str
    recipe_description: str
    recipe_ingredients: list[str]

For this example you want a list of `Recipe` objects, so pass `list[Recipe]` to the `response_schema` field of the `config`.

In [41]:
result = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few imaginative cookie recipes along with a one-sentence description as if you were a gourmet restaurant and their main ingredients',
    config={
        'response_mime_type': 'application/json',
        'response_schema': Recipe,
    },
 )


In [42]:
result.parsed
print(result.text)

{
  "recipe_name": "Celestial Starburst Cookies",
  "recipe_description": "A cosmic blend of white chocolate and passion fruit that bursts with stardust shimmer.",
  "recipe_ingredients": ["White chocolate", "Passion fruit puree", "Edible silver glitter", "Almond flour", "Star anise"]
}


It is the recommended method if you're using a compatible model.

## Next Steps
### Useful API references:

Check the [structured ouput](https://ai.google.dev/gemini-api/docs/structured-output) documentation or the [`GenerationConfig`](https://ai.google.dev/api/generate-content#generationconfig) API reference for more details

### Related examples

* The constrained output is used in the [Text summarization](../examples/json_capabilities/Text_Summarization.ipynb) example to provide the model a format to summarize a story (genre, characters, etc...)
* The [Object detection](../examples/Object_detection.ipynb) examples are using the JSON constrained output to uniiformize the output of the detection.

### Continue your discovery of the Gemini API

JSON is not the only way to constrain the output of the model, you can also use an [Enum](../quickstarts/Enum.ipynb). [Function calling](../quickstarts/Function_calling.ipynb) and [Code execution](../quickstarts/Code_Execution.ipynb) are other ways to enhance your model by either using your own functions or by letting the model write and run them.